# Evaluation of wind data (wind)

> functions for working with and converting wind data

Equations in this section are the equivalent to the equations in appendix E of ITTC. However, mostly the functions here are a wrapper over the functions found in the `trig` module

Remember that windspeed is ALWAYS positive, the direction tells you if it is helping or hindering the ship. Negative wind speeds are positive windspeeds rotated $180^\circ$.

In [ ]:
#| default_exp wind

In [ ]:
#| hide
from nbdev.showdoc import *


In [ ]:
#| export
import numpy as np
import pandas as pd
from fastcore.test import *
from pyseatrials.trig import *


## True Wind

The two functions in this section convert relative wind speed and direction to thier True equivalents.

### Speed

Calculating true windspeed from relative windspeed and direction

$$V_\textrm{WR} = \sqrt{V_{WR}^2 + V_G^2 - 2V_{WR} V_G \cos(\psi_{WR})}$$

**ITTC equations**: E-2


In [ ]:
#| export
def rel2true_speed(relative_windspeed:float, #speed of wind relative to ship
                            sog:float, #speed over ground
                           relative_wind_direction:float #wind direction relative to ship
                           ) ->float: # True windspeed
    "converts a relative wind speed and direction to a true wind speed"
    return law_of_cosines(relative_windspeed, sog, relative_wind_direction)

If the wind is blowing has a relative angle of 0, (meaning the wind is blowing from the bow to the stern) and has a relative windspeed of 2 knots, then the true windspeed can be calculated as

In [ ]:
rel2true_speed(2, 20, 0)

18.0

Conversely if the wind is blowing from stern to bow this means the true windspeed is 22

In [ ]:
rel2true_speed(2, 20, np.pi)

22.0

However if the relative angle is $-45^\circ$ then the true windspeed is

In [ ]:
rel2true_speed(2, 20, -np.pi/4)

18.63951333874026

Remember that windspeed is ALWAYS positive, the direction tells you if it is helping or hindering the ship

In [ ]:
#| hide
test_eq(rel2true_speed(2, 20, 0), 18) #

### Direction

How to calculate the true direction from the relative direction.

$$ \psi_\textrm{WT} = \text{arctan2}\left( \frac{  
V_\textrm{WR} \textrm{sin}(\psi_\textrm{WR} + \psi)  - V_\textrm{G}\textrm{sin}(\psi)}{  
V_\textrm{WR} \textrm{cos}(\psi_\textrm{WR} + \psi)  - V_\textrm{G}\textrm{cos}(\psi)} \right)

$$

**ITTC equations**: E-3

In [ ]:
#| export
def rel2true_dir(
    relative_wind_speed:float, #Speed of the wind relative to the ship
    sog:float, #Speed of the ship overground
    relative_wind_direction:float, #direction of wind in radians relative to the ship
    vessel_heading:float, #The direction of the ship through the water
    constrain_to_positive:bool = True #Should the function return a value between 0 and 2 pi
)-> float: # the true wind directionangle relative to north in radians.
    "converts relative wind direction to true wind direction"
    #This function should probably be abstracted to a more trignonometric form
    numerator = opposite_magnitude_fn(relative_wind_speed, relative_wind_direction + vessel_heading) - opposite_magnitude_fn(sog, vessel_heading) 
    denominator = adjacent_magnitude_fn(relative_wind_speed, relative_wind_direction + vessel_heading) - adjacent_magnitude_fn(sog, vessel_heading) 
    
    gamma = np.arctan2(numerator, denominator)
        
    #prevents negative angles if constrain to positive is true
    #using this method instead of an if statement means the function can perform vectorised operations
    gamma = gamma + 2*np.pi*(gamma<0)*constrain_to_positive
    
    return gamma 
    

Considering a ship where the relative wind direction is 0, the relative windspeed is 2, and the ships heading is $45^\circ$ or $\frac{\pi}{2}$, then the true wind speed is

In [ ]:
rel2true_dir(2,20,0,2)

5.141592653589793

Although by default the wind direction is constrained to return only positive values between $0$ and $2\pi$, values between $-\frac{\pi}{2}$ and  $\frac{\pi}{2}$, can be created using `constrain_to_positive = False`

In [ ]:
rel2true_dir(2,20,0,2, False)

-1.1415926535897933

In [ ]:
#### bug alert!
#does this make sense?
#should it be 180 or zero?
rel2true_dir(2,20,0,0, False)

3.141592653589793

In [ ]:
#| hide
test_close(rel2true_dir(2,20,0,2), 5.14159, eps=1e-03)

## Relative Wind

### Speed

Calculating relative wind speed from true wind speed and direction as shown in the equation below

$$V_\textrm{WR} = \sqrt{V_{TW}^2 + V_G^2 + 2V_{TW} V_G \cos(\psi_{WT} - \psi)}$$

**ITTC equations**: E-7, E-10

In [ ]:
#| export
def true2rel_speed(true_wind_speed:float, #The windspeed over ground
                            sog:float, #Speed over ground of the vessel
                            true_wind_direction:float, #Direction of wind relative to north
                            vessel_heading:float, #Direction of vessel in water relative to north
)-> float: #returns relative windspeed using the same units entered
    
    "converts true windspeed to relative"
    
    return np.sqrt(true_wind_speed**2 +sog**2 +2*true_wind_speed*sog*np.cos( true_wind_direction - vessel_heading))
           

In [ ]:
#|hide
#The previous version of the function looked like this
#However this version lead to non-trivial rounding errors. 
#This version is nicer as you can use a law of cosines directly as a backend but until we have time to de-bug properly just using the ITTC formulation 
#get precise and consistant answers
"""
def true2rel_speed(true_wind_speed:float, #The windspeed over ground
                            sog:float, #Speed over ground of the vessel
                            true_wind_direction:float, #Direction of wind relative to north
                            vessel_heading:float, #Direction of vessel in water relative to north
)-> float: #returns relative windspeed using the same units entered
    
    "converts true windspeed to relative"
    
    return law_of_cosines(true_wind_speed, sog, true_wind_direction - vessel_heading +np.pi)


"""

'\ndef true2rel_speed(true_wind_speed:float, #The windspeed over ground\n                            sog:float, #Speed over ground of the vessel\n                            true_wind_direction:float, #Direction of wind relative to north\n                            vessel_heading:float, #Direction of vessel in water relative to north\n)-> float: #returns relative windspeed using the same units entered\n    \n    "converts true windspeed to relative"\n    \n    return law_of_cosines(true_wind_speed, sog, true_wind_direction - vessel_heading +np.pi)\n\n\n'

As an example consider a ship travelling at 20 knots with a direction due north. The wind blows due north with a speed of 22 knots

In [ ]:
true2rel_speed(22,20, 0,0)

42.0

In [ ]:
true2rel_speed(22,20, 0, np.pi)

2.0

In [ ]:
#| hide
#vessel heading with the wind
test_eq(true2rel_speed(22,20, 0,0), 42)
#vessel heading into the wind
test_eq(true2rel_speed(22,20, 0,np.pi), 2)


##
## This test ensures that the relative wind speed is constant for 360 degs in other words ensures that rel2true then true2rel returns the original value
##
test = pd.DataFrame({'rel_degs':np.linspace(1,360, 360)})

sog = 20
heading = 0
rel_wind_Speed = 10

test['rel_rads'] = np.deg2rad(test['rel_degs'])

test['true_rads'] = rel2true_dir(rel_wind_Speed , sog, test['rel_rads'], heading)
#test['true_rads_constr'] = rel2true_dir(rel_wind_Speed , sog, test['rel_rads'], heading, constrain_to_positive = True)
test['true_wind_speed'] = rel2true_speed(rel_wind_Speed, sog, test['rel_rads'])

test['rel_wind_speed'] = true2rel_speed(true_wind_speed = test['true_wind_speed'] ,
                            sog = sog, 
                            true_wind_direction = test['true_rads'], 
                            vessel_heading = heading 
)

#ensures that all the values are equal to 10, uses round as there may be very small rounding errors

test_eq(np.all(np.equal(np.round(test['rel_wind_speed'],4), 10)), True)


### Direction

Caclulating relative windspeed from true windspeed and direction


$$\psi_\textrm{WR} = \text{arctan2}\left( \frac{ V_\textrm{WT} \textrm{sin}(\psi_\textrm{WT} - \psi)}{ V_\textrm{G} + V_\textrm{WT} \textrm{cos}(\psi_\textrm{WT} - \psi)} \right)$$


**ITTC equations**: E-6, E-9

In [ ]:
#| export
def true2rel_dir(
                            true_wind_speed:float, #The windspeed over ground
                            sog:float, #Speed over ground of the vessel
                            true_wind_direction:float, #Direction of wind relative to north
                            vessel_heading:float, #Direction of vessel in water relative to north
                            constrain_to_positive:bool = True #Should the function return a value between 0 and 2 pi
)-> float: #relative wind direction
    
    "converts true direction speed to relative"
    
    gamma  = find_gamma_fn(true_wind_speed, sog, true_wind_direction - vessel_heading)

    return gamma + 2*np.pi*(gamma<0)*constrain_to_positive

We can see that if a ship is heading due north and the true windspeed is also due east. When the ship is at 20 knots and the wind is at 22 knots, the relative direction of the wind is as below

In [ ]:
true2rel_dir(22,20, np.pi/2,0, constrain_to_positive=False)

0.8329812666744317

If the wind direction is due west, the relative wind direction is the negative of the previous value

In [ ]:
true2rel_dir(22,20, -np.pi/2,0, constrain_to_positive=False)

-0.8329812666744317

Unless of course 'constrain_to_positive' is set to true

In [ ]:
true2rel_dir(22,20, -np.pi/2,0, constrain_to_positive=True)


5.450204040505154

In [ ]:
#| hide
test_eq(true2rel_dir(20,22, 0,0), 0)
#test_eq(round(true2rel_dir(22,20, 0,0),5), round(np.pi, 5))

## Average wind speed and direction across a double run

The ITTC method proposes taking the average windspeed across a double run to average for errors in readings
This function is a wrapper round the 'combine_vectors' function from the `trig` module.
$$c \; \text{cos}(\gamma) = a \; \text{cos}(\alpha) + b \; \text{cos}(\beta), $$
$$c \; \text{sin}(\gamma) = a \; \text{sin}(\alpha) + b \; \text{sin}(\beta), $$

$$c = \sqrt{(\frac{c \; \text{cos}(\gamma)^2 + c \; \text{sin}(\gamma)^2}{2})} $$

$$\gamma = \text{arctan2} \left( \frac{c \; \text{cos}(\gamma)}{c \; \text{sin}(\gamma)} \right)$$

Where $a$ and $b$ are the true windspeeds of two paired runs and $\alpha$ and $\beta$ are there true direction. 

The function takes the magnitude and angle of two vectors and outputs the magnitude and angle of the resultant vector.

One can question whether this approach makes physical sense under certain conditions, the user should consider what is happening in the test and consult the reasoning of the ITTC for further details.

**ITTC equations**: E-4, E-5

In [ ]:
#| export
def double_run_average(a, b, alpha, beta):
    #it makes no difference if a/2, b/2 is used or average_velocity/2 the result is the same
    average_velocity, average_direction = combine_vectors(a, b, alpha, beta)

    return average_velocity/2, average_direction

In the example below on the first run a ship is faced with a wind of 13m/s blowing from the north, whilst on the second run the wind has dropped to 5m/s and is coming from the east. The function calculates the mean wind speed across both runs. 

In [ ]:

wind_speed, wind_direction = double_run_average(a = 13, b = 5, alpha = 0, beta = 1.6)

print("The wind speed is {0} m/s, the wind direction is {1} radians".format(round(wind_speed, 2), round(wind_direction, 2)))

The wind speed is 6.9 m/s, the wind direction is 0.37 radians


## Correcting for the height of the Anemometer

Adjusts the windspeed taking into account the height of the anemometer on the ship relative to the reference height for windspeed

$$V_\textrm{WTref} = V_\textrm{WT} \left( \frac{Z_\textrm{ref}}{Z_a} \right)^\frac{1}{9}$$

**ITTC equations**: E-8

In [ ]:
#| export
def vertical_position_anemometer(true_wind_speed:float, #True windspeed [m/s]
                                 reference_height:float, #reference height [m]
                                 measured_height:float  # measured height [m]
                                )-> float: #The true windspeed corrected for measurement height
    
    "Adjusts the windspeed taking into account the height of the anemometer on the ship relative to the reference height for windspeed"

    
    return true_wind_speed * (reference_height/measured_height)**(1/9)

The adjusted wind speed differences are often small but the corrections can still influence the final result

In [ ]:
vertical_position_anemometer(22,5, 10)

20.36924367032039

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()